In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

# paths
dir_raw   = Path("raw_data")
file_txt  = dir_raw / "airsim_rec.txt"
pcd_dir   = dir_raw / "lidar"      # folder with per-step LiDAR .npy files
out_csv   = dir_raw / "airsim_trajectory.csv"

In [2]:
# read tab-delimited AirSim log
df = pd.read_csv(file_txt, sep=r"\t+", engine="python", comment="#")
# drop quaternion, orientation, image, and timestamp columns entirely
df.drop(columns=["Q_W","Q_X","Q_Y","Q_Z","VehicleName","ImageFile","TimeStamp","TimeSec"], errors="ignore", inplace=True)

In [3]:
import torch
import torch.nn as nn
import numpy as np


class PointNetFeat(nn.Module):
    def __init__(self, out_dim=64):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(3, 64), nn.ReLU(),
            nn.Linear(64, 128), nn.ReLU(),
            nn.Linear(128, out_dim)
        )
    def forward(self, x):  # x: (B, N, 3)
        f = self.mlp(x)        # (B, N, out_dim)
        return f.max(dim=1).values  # (B, out_dim)

# instantiate once and freeze
pointnet = PointNetFeat(out_dim=64)
pointnet.eval()
for p in pointnet.parameters(): p.requires_grad = False

# allocate arrays for new features
lidar_feats = np.zeros((len(df), 64), dtype=np.float32)

# iterate rows and compute per-scan embedding
for idx in df.index:
    pcd_file = pcd_dir / f"pc_{idx:06d}.npy"
    if pcd_file.exists():
        pts = np.load(pcd_file).astype(np.float32)  # (N,3)
        with torch.no_grad():
            emb = pointnet(torch.from_numpy(pts).unsqueeze(0))  # (1,64)
        lidar_feats[idx] = emb.squeeze(0).numpy()
    # else leave zeros

# attach each of the 64 dims as separate DataFrame columns
for i in range(64):
    df[f"LIDAR_F{i}"] = lidar_feats[:, i]

In [4]:
final_cols = [
    "POS_X","POS_Y","POS_Z",
    "VEL_X","VEL_Y","VEL_Z"
] + [f"LIDAR_F{i}" for i in range(64)]
# sanity-check
missing = [c for c in final_cols if c not in df.columns]
if missing:
    raise RuntimeError(f"Missing columns after preprocessing: {missing}")

# save
pd.DataFrame(df[final_cols]).to_csv(out_csv, index=False)
print(f"[✓] Saved {len(df)} rows with columns: {final_cols}")

[✓] Saved 54000 rows with columns: ['POS_X', 'POS_Y', 'POS_Z', 'VEL_X', 'VEL_Y', 'VEL_Z', 'LIDAR_F0', 'LIDAR_F1', 'LIDAR_F2', 'LIDAR_F3', 'LIDAR_F4', 'LIDAR_F5', 'LIDAR_F6', 'LIDAR_F7', 'LIDAR_F8', 'LIDAR_F9', 'LIDAR_F10', 'LIDAR_F11', 'LIDAR_F12', 'LIDAR_F13', 'LIDAR_F14', 'LIDAR_F15', 'LIDAR_F16', 'LIDAR_F17', 'LIDAR_F18', 'LIDAR_F19', 'LIDAR_F20', 'LIDAR_F21', 'LIDAR_F22', 'LIDAR_F23', 'LIDAR_F24', 'LIDAR_F25', 'LIDAR_F26', 'LIDAR_F27', 'LIDAR_F28', 'LIDAR_F29', 'LIDAR_F30', 'LIDAR_F31', 'LIDAR_F32', 'LIDAR_F33', 'LIDAR_F34', 'LIDAR_F35', 'LIDAR_F36', 'LIDAR_F37', 'LIDAR_F38', 'LIDAR_F39', 'LIDAR_F40', 'LIDAR_F41', 'LIDAR_F42', 'LIDAR_F43', 'LIDAR_F44', 'LIDAR_F45', 'LIDAR_F46', 'LIDAR_F47', 'LIDAR_F48', 'LIDAR_F49', 'LIDAR_F50', 'LIDAR_F51', 'LIDAR_F52', 'LIDAR_F53', 'LIDAR_F54', 'LIDAR_F55', 'LIDAR_F56', 'LIDAR_F57', 'LIDAR_F58', 'LIDAR_F59', 'LIDAR_F60', 'LIDAR_F61', 'LIDAR_F62', 'LIDAR_F63']


In [5]:
print(pd.read_csv(out_csv).head())
print("Columns:", pd.read_csv(out_csv).columns.tolist())

   POS_X  POS_Y      POS_Z  VEL_X  VEL_Y     VEL_Z  LIDAR_F0  LIDAR_F1  \
0    0.0    0.0 -29.586658    0.0    0.0 -0.004825       0.0       0.0   
1    0.0    0.0 -29.588932    0.0    0.0 -0.011343       0.0       0.0   
2    0.0    0.0 -29.590113    0.0    0.0 -0.012474       0.0       0.0   
3    0.0    0.0 -29.591423    0.0    0.0 -0.013139       0.0       0.0   
4    0.0    0.0 -29.592745    0.0    0.0 -0.013401       0.0       0.0   

   LIDAR_F2  LIDAR_F3  ...  LIDAR_F54  LIDAR_F55  LIDAR_F56  LIDAR_F57  \
0       0.0       0.0  ...        0.0        0.0        0.0        0.0   
1       0.0       0.0  ...        0.0        0.0        0.0        0.0   
2       0.0       0.0  ...        0.0        0.0        0.0        0.0   
3       0.0       0.0  ...        0.0        0.0        0.0        0.0   
4       0.0       0.0  ...        0.0        0.0        0.0        0.0   

   LIDAR_F58  LIDAR_F59  LIDAR_F60  LIDAR_F61  LIDAR_F62  LIDAR_F63  
0        0.0        0.0        0.0      